In [3]:
import warnings
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm
import pandas as pd

In [4]:
PLOT_SHOW=True
PLOT_TYPE = False

time_period = 100
max_iter = 1e6

labels = ['Close','Open','High','Low']


def calc_mape(predicted_data, true_data):
    return np.divide(np.sum(np.divide(np.absolute(predicted_data - true_data), true_data), 0), true_data.shape[0])

In [8]:
dataset =  pd.read_excel('ExchangeRates.xlsx', sheet_name='usd-cad')[labels]
dataset = np.array(dataset)

In [1]:
def get_best_num_states(dataset, max_range = 15, time_period= 50, max_iter=1e6):
    likelihoods = np.empty(13)
    aics = np.empty(13)
    bics = np.empty(13)

    for num_states in range(2,max_range):
        
        num_params = num_states**2 + 2*num_states - 1
        
        model = hmm.GaussianHMM(n_components=num_states, covariance_type='full', tol=1e-6, n_iter=max_iter)
        model.fit(dataset[100:,:])
        
        if model.monitor_.iter == max_iter:
            print('max iterations reached. Error')
            sys.exit(1)
        
        likelihoods[num_states-2] = model.score(dataset)
        aics[num_states-2] = -2 * model.score(dataset) + 2 * num_params
        bics[num_states-2] =  -2 * model.score(dataset) +  num_params * np.log(dataset.shape[0])

    opt_states = np.argmin(bics) + 2

    return opt_states

In [9]:
likelihoods = np.empty(13)
aics = np.empty(13)
bics = np.empty(13)

for num_states in range(2,15):
        num_params = num_states**2 + num_states
        model = hmm.GaussianHMM(n_components=num_states, covariance_type='full', tol=0.0001, n_iter=max_iter)
        model.fit(dataset[time_period:,:])
        if model.monitor_.iter == time_period:
            print('Increase number of iterations')
            sys.exit(1)
        likelihoods[num_states-2] = model.score(dataset)
        aics[num_states-2] = -2 * model.score(dataset) + 2 * num_params
        bics[num_states-2] =  -2 * model.score(dataset) +  num_params * np.log(dataset.shape[0])
    
opt_states = np.argmin(bics) + 2
print('Optimum number of states are {}'.format(opt_states))

<class 'int'>


TypeError: 'float' object cannot be interpreted as an integer

In [27]:
predicted_stock_data = np.empty([0,dataset.shape[1]])


Number of states is  13


In [ ]:

for idx in reversed(range(time_period)):
    train_dataset = dataset[idx + 1:,:]
    test_data = dataset[idx,:]; 
    num_examples = train_dataset.shape[0]
    if idx == time_period - 1:
        model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=0.0001, n_iter=max_iter, init_params='stmc')
    else:
        # Retune the model by using the HMM paramters from the previous iterations as the prior
        model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=0.0001, n_iter=max_iter, init_params='')
        model.transmat_ = transmat_retune_prior 
        model.startprob_ = startprob_retune_prior
        model.means_ = means_retune_prior
        model.covars_ = covars_retune_prior

    model.fit(np.flipud(train_dataset))

    transmat_retune_prior = model.transmat_
    startprob_retune_prior = model.startprob_
    means_retune_prior = model.means_
    covars_retune_prior = model.covars_

    if model.monitor_.iter == max_iter:
        print('Increase number of iterations')
        sys.exit(1)


    iters = 1
    past_likelihood = []
    curr_likelihood = model.score(np.flipud(train_dataset[0:K - 1, :]))
    while iters < num_examples / K - 1:
        past_likelihood = np.append(past_likelihood, model.score(np.flipud(train_dataset[iters:iters + K - 1, :])))
        iters = iters + 1
    likelihood_diff_idx = np.argmin(np.absolute(past_likelihood - curr_likelihood))
    predicted_change = train_dataset[likelihood_diff_idx,:] - train_dataset[likelihood_diff_idx + 1,:]
    predicted_stock_data = np.vstack((predicted_stock_data, dataset[idx + 1,:] + predicted_change))


mape = calc_mape(predicted_stock_data, np.flipud(dataset[range(100),:]))

if PLOT_TYPE:
    hdl_p = plt.plot(range(100), predicted_stock_data)
    plt.title('Predicted stock prices')
    plt.legend(iter(hdl_p), ('Close','Open','High','Low'))
    plt.xlabel('Time steps')
    plt.ylabel('Price')
    plt.figure()
    hdl_a = plt.plot(range(100),np.flipud(dataset[range(100),:]))
    plt.title('Actual stock prices')
    plt.legend(iter(hdl_p), ('Close','Open','High','Low'))
    plt.xlabel('Time steps')
    plt.ylabel('Price')
else:
    for i in range(4):
        plt.figure()
        plt.plot(range(100), predicted_stock_data[:,i],'k-', label = 'Predicted '+labels[i]+' price')
        plt.plot(range(100),np.flipud(dataset[range(100),i]),'r--', label = 'Actual '+labels[i]+' price')
        plt.xlabel('Time steps')
        plt.ylabel('Price')
        # plt.title(labels[i]+' price'+ ' for '+stock[:-4])
        plt.grid(True)
        plt.legend(loc = 'upper left')
    